In [3]:
import cv2
import numpy as np
import imageio as io
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize,StandardScaler
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
import skimage.io as io
import math
from operator import eq
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from skimage import color
from operator import eq
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import rbf_kernel
import math

In [4]:
#FER:Face Emomtion Recognition model Using Dynamic Local Ternary Patterns With Kernel Extreme Learning Machine Classifier
# In this model I use GCN to enhance input image as preprocessing step, DLTP feature descriptor, 
# PCA to reduce the high-dimensional DLTP features 
# and  K-ELM classifier to classify the face expression

In [5]:
#Functional Description
# •	Cropping:
# This function crop input image with size 48x48
# •	Convertion:
# Convert the cropped image into grayscale
# •	Global contrast normalization:
# Enhance the grayscale image using GCN techniquies
# •	Dynamic Local Ternary Pattern:
# Get the ternary pattern of enhanced image using dynamic threshold value for every image and this ternary pattern is splited into two binary code P_DLTP & N_DLTP
# •	Positive dynamic Local Ternary Pattern:
# Binary threshold function for positive pattern and get the weighted sum of it
# •	Negative dynamic Local Ternary Pattern:
# Binary threshold function for negative pattern and get the weighted sum of it
# •	Build the sub histogram :
# the histograms are computed separately and the result is concatenated together
# •	Build K-ELM  classifier:
# The K-ELM classifier uses kernels that maps the features into
# higher dimensional space and we use  the Gaussian kernels 

In [6]:
# The pipeline consists of five units, namely preprocessing, image enhancement,feature extraction ,
#dimensionality reduction and classification

In [7]:
le = preprocessing.LabelEncoder()
#random_seed parameter in split data function
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)
#control the width of gaussian kernel
sigma=0.0008
# regularization parameter
c=64
# regularization parameter
reg=100
# small constant error to avoid computation errors €
err=1e-4

In [8]:
#return the quantized value of the surrounding neighbors of current pixel
def DLTP(ic,arr1):
    #the intensity value of the center pixel
    Ic=ic
    #8 neighbour of center pixel
    arr=arr1[0]
    #loop over 8 neighbour
    for i in range(len(arr)-1):
        #the intensity value of one neighbour pixel
        In=arr[i]
        #calculate the tao "The threshold" automatically
        tao=abs(In-Ic)/(Ic+0.0001)
        #Neighbor pixels that falls above Ic+tao to 1
        if(In>Ic+tao):
            arr[i]=1
        #Neighbor pixels that falls below Ic-tao to -1    
        elif(In<=Ic-tao):
            arr[i]=-1
        #Neighbor pixels that falls in between Ic+tao and Ic-tao are quantized to 0    
        else:
            arr[i]=0
    return arr        

In [9]:
#return the postive DLTP pattern,by putting value of 1 in the postive places and otheres will be 0 
def Pos_DLTP(arr):
    pos_arr=np.zeros((1,8), np.uint8)[0]
    k=0
    for i in arr:
        if (i==1):
            pos_arr[k]=1
        k=k+1    
    return  pos_arr       

In [10]:
#return the negative DLTP pattern,by putting value of 1 in the negative places and otheres will be 0 
def Neg_DLTP(arr):
    neg_arr=np.zeros((1,8), np.uint8)[0]
    k=0
    for i in arr:
        if (i==-1):
            neg_arr[k]=1
        k=k+1     
    return  neg_arr       

In [11]:
# The resulting negative and positive binary patterns
# are then multiplied with fixed weights and are summed up
#to return DLTP encoded values
def Convertion(arr):
    value=0
    arr_value=[1,2,4,8,16,32,64,128]
    value=np.dot(arr,arr_value)
    return value

In [12]:
#pad the current block with specific size
def padding(block,size):
    padd=np.pad(block, (size, size), 'constant', constant_values=(0, 0))
    return padd

In [13]:
#return 8 neighbour of current pixel
def GetArray(block,x,y):
    one=block[x-1][y+1]
    two=block[x][y+1]
    three=block[x+1][y+1]
    four=block[x+1][y]
    five=block[x-1][y+1]
    six=block[x+1][y-1]
    seven=block[x][y-1]
    eight=block[x-1][y-1]
    zero=block[x][y]
    arr=[]
    arr.append([one,two,three,four,five,six,seven,eight])
    return zero,arr

In [14]:
#Get both negative and positive DLTP histogram
def DLTP_Hist(block):
    img=padding(block,1)
    w,l=img.shape
    tlbphist=[]
    neg_hist=[]
    pos_hist=[]
    #loop for every pixels in block
    for y in range(1,w-1):
        for x in range(1,l-1):
            #return current pixel and 8 neighbour of current pixel
            gc,array=GetArray(img,x,y)
            #return DLTP code [1,0,-1]
            array2=array1=DLTP(gc,array)
            #the generated quantized value is further divided into negative and positive patterns
            #get the negative and positive DLTP code
            neg_arr=Neg_DLTP(array1)
            pos_arr=Pos_DLTP(array2)
            #calculate the weighted sum function for positive pattern and negative pattern to get PDLTP  and NDLTP
            neg_hist.append(Convertion(neg_arr))
            pos_hist.append(Convertion(pos_arr))
    #calculate both pos and neg histogram separately        
    (histpos, _) = np.histogram(np.array(pos_hist),bins=np.arange(0,255),range=(0,255))
    histpos = histpos.astype("float")
    (histneg, _) = np.histogram(np.array(neg_hist),bins=np.arange(0,255),range=(0,255))
    histneg = histneg.astype("float")
    return np.array(histneg).flatten(),np.array(histpos).flatten()

In [15]:
# •	Properties of DLTP :
# o	It is very useful in extracting facial texture information
# o	Overcomes the manual determination of threshold in the traditional LTP descriptor.
# o	The threshold in DLTP is automatically determined using local neighborhood pixel intensities 
# o	It dynamically updates the threshold depending on the pixel intensity values
#______________________________________________________________________________________________


#Algorithm to extract the DLTP features
def Algorithm_DLTP(img1,blocksize=4):
    #get the width abd length of input image
    w,l=img1.shape
    #divided image into NxN blocks
    window=w//blocksize
    #two arrays to store both negative and positive DLTP histogram for all blocks
    Ndltp_hist=[]
    Pdltp_hist=[]
    #pad image with 2 cells
    img=padding(img1,2)
    #loop over each block
    for r in range(2,w-2,window):
        for c in range(2,l-2,window):
            #extract the block
            block = img[r:r+window,c:c+window]
            #get both positive and negative DLTP histogram
            hneg,hpos= DLTP_Hist(block)
            #append every positive and negative DLTP hist to two separate arrays
            Pdltp_hist.append(np.array(hpos))
            Ndltp_hist.append(np.array(hneg))
    #array that store all positive and negative DLTP hist        
    dltpfeature=[]        
    #concatenate both histograms to get final features
    dltpfeature.append(np.array(Pdltp_hist).flatten())
    dltpfeature.append(np.array(Ndltp_hist).flatten())
    return np.array(dltpfeature).flatten()

In [16]:
#Extract DLTP features
def feature_DLTP(imgs):
    #process the image
    gray=preprocessing1(imgs)
    #enhance the image using GCN
    enhanced=global_contrast_normalization(gray,reg,err)
    #normalize the image
    norm=min_max_normalize(enhanced)
    return np.array(Algorithm_DLTP(norm,4))         

In [17]:
#process the image
def preprocessing1(img):
    #resize the input image
    resized = cv2.resize(img,(48,48), interpolation = cv2.INTER_AREA)
    #convert image to gray image
    gray=color.rgb2gray(resized)
    return gray

In [18]:
#load dataset:loop over all files of emotion
def load_dataset():
    features = []
    labels = []
    for i, fn in enumerate(img_filenames):
            #get the name of directory and store it as label of image
            label = fn.split('.')[0]
            #append the path of dataset and the new folder
            path = os.path.join(path_to_dataset, fn)
            #get all directory of this path
            subpath=os.listdir(path)
            #loop over these diroctories
            for k, d in enumerate(subpath):
                #append the label of image to array
                labels.append(label)
                #get the path of image
                imgpath = os.path.join(path, d)
                #read the image
                imgs = io.imread(imgpath)
                #get the DLTP feature of image then store it in features array
                features.append(feature_DLTP(imgs))
                #print this every 10 images
                if k > 0 and k % 10== 0:
                    print("[INFO] processed {}/{}".format(k, len(subpath)))
    return features, labels            

In [19]:
def min_max_normalize(img):
    return ((img-min(img.flatten()))/(max(img.flatten())-min(img.flatten())))*255

In [20]:
#To enhance image according to illumination:
    # o	subtracts each pixel from its mean pixel value
    # o	divides the mean subtracted pixels by their standard deviation 
    # o	add regularization parameter to the standard deviation  λ
    # o	add small constant error to avoid computation errors €

def global_contrast_normalization(image1,reg,err):
    image=np.array(image1)
    mean=np.mean(image)
    image=(image-mean)/(max(err,math.sqrt((reg)+np.mean((image-mean)**2))))
    return image

In [21]:
def split_data(features,labels,precentage):
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    return train_features, test_features, train_labels, test_labels

In [22]:
# encode the string labels into codes  
def encode(labels):
    le = preprocessing.LabelEncoder()
    return le.fit_transform(labels)

In [23]:
import matplotlib.pyplot as plt
def show_image(im):
    plt.imshow(im,cmap="gray")
    plt.show()

In [54]:
path_to_dataset =r'CK+48'
img_filenames = os.listdir(path_to_dataset)

In [55]:
#Visualize the dataset

In [56]:
predictlist=img_filenames = os.listdir(path_to_dataset)
print(img_filenames)

['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']


In [27]:
print("Label        : "+"        Count")
print("===================================")
for i, fn in enumerate(img_filenames):
        label = fn.split('.')[0]
        path = os.path.join(path_to_dataset, fn)
        subpath=os.listdir(path)
        
        i=len(subpath)
        print(label+"                 ",i)
        print("___________________________________")

Label        :         Count
anger                  135
___________________________________
contempt                  54
___________________________________
disgust                  177
___________________________________
fear                  75
___________________________________
happy                  207
___________________________________
sadness                  84
___________________________________
surprise                  249
___________________________________


In [28]:
#Loading the data and extract the features and labels 

In [29]:
print('Loading dataset. This will take time ...')
features, labels = load_dataset()
print('Finished loading dataset.')

Loading dataset. This will take time ...
[INFO] processed 10/135
[INFO] processed 20/135
[INFO] processed 30/135
[INFO] processed 40/135
[INFO] processed 50/135
[INFO] processed 60/135
[INFO] processed 70/135
[INFO] processed 80/135
[INFO] processed 90/135
[INFO] processed 100/135
[INFO] processed 110/135
[INFO] processed 120/135
[INFO] processed 130/135
[INFO] processed 10/54
[INFO] processed 20/54
[INFO] processed 30/54
[INFO] processed 40/54
[INFO] processed 50/54
[INFO] processed 10/177
[INFO] processed 20/177
[INFO] processed 30/177
[INFO] processed 40/177
[INFO] processed 50/177
[INFO] processed 60/177
[INFO] processed 70/177
[INFO] processed 80/177
[INFO] processed 90/177
[INFO] processed 100/177
[INFO] processed 110/177
[INFO] processed 120/177
[INFO] processed 130/177
[INFO] processed 140/177
[INFO] processed 150/177
[INFO] processed 160/177
[INFO] processed 170/177
[INFO] processed 10/75
[INFO] processed 20/75
[INFO] processed 30/75
[INFO] processed 40/75
[INFO] processed 50/

In [30]:
train_features, test_features, train_labels, test_labels=split_data(np.array(features),np.array(labels),0.2)

In [31]:
print(np.array(train_features).shape)

(784, 8128)


In [32]:
#PCA

In [33]:
# In this step I want to reduce the number of input features. 
# As High-dimensional features affect the performance of classifiers,

# features from DLTP have high-dimensional which contain a lot of redundant information.
# Too many features slow down the classification process and lead to degradation in the accuracy of the classifier. 
# So, I use Principal component analysis (PCA) in this step

In [34]:
pca = PCA(n_components=0.7)
X_train1 = pca.fit_transform(train_features)
X_test1 = pca.transform(test_features)
train_features=X_train1
test_features=X_test1
train_labels=train_labels
test_labels=test_labels

In [35]:
print(np.array(train_features).shape)

(784, 14)


In [36]:
#K-ELM

In [37]:
# The K-ELM classifier is the kernalized variant of the extreme learning machine (ELM) classifier;
# The classical ELM is a single-layer Feed-forward neural network classifier which requires a large number of hidden nodes,
# those results in higher computational complexity and a longer training time, so I improve this classifier and 
# introduce a new variation of ELM classifier which is K-ELM classifier
# •K-ELM classifier:
# It uses kernels that map the features into higher dimensional space. Also, the RBF kernels required is 
# much less in K-ELM than the hidden nodes in a conventional ELM classifier.
# •	kernel technique:
# It states that given two input vectors xi and xj, 
# the dot product of their mapped features represented by h(xi) . h(xj) can be replaced by a kernel function Ω(xi; xj).


In [38]:
def kernel(X, Y=None,g=None):
    k=rbf_kernel(X, Y, gamma=g)
    return k

In [39]:
#Input:
# 1/sigma : control the width of gaussian kernel [gamma dec underfitting] [gamma inc overfitting]
#c : regularization parameter ,It aids in improving the generalization performance of the classifier 
# x , y : two input vectors training and test
#Output : Calculate the output weight of KELM network ß

def KELM_Beta(sigma,c,x,y):
    #convert to numrical values
    le.fit(y)
    y = le.transform(y)
    #count the emotions
    emotion = len(np.unique(y))
    #calculate One_Hot_Encoding
    One_Hot_Encoding=np.zeros((emotion,emotion), np.uint8)
    #loop over the emotions
    for em in range (emotion):
        One_Hot_Encoding[em][em]=1
    #map from numrical values to One_Hot_Encoding    
    T = One_Hot_Encoding[y, :]    
    N,d=x.shape
    #Define the kernel matrix of K-ELM = h(xi) . h(xj)
    Omega=kernel(x,None,sigma)
    #get indintity matrix of lenght N
    I=np.eye(N)
    #Calculate the output weight of KELM network ß
    Beta = np.linalg.inv((I /c) + Omega).dot(T)    
    return Beta,One_Hot_Encoding

In [40]:
#Input:
# 1/sigma : control the width of gaussian kernel [gamma dec underfitting] [gamma inc overfitting]
# x , x_test : two input vectors training and test
#Output : output the class that has the maximum magnitude

def KELM_Output(sigma,x,x_test,Beta,One_Hot_Encoding):
    hx=kernel(x_test,x,sigma)
    #output nodes in vectroized form
    fx= hx.dot(Beta)
    #the classifier selects the output node that has the maximum magnitude as the output class
    p=np.argmax(fx,axis=1)
    return p

In [41]:
def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)    
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(y_pred)))
    size=len(rt)
    acc=(eqw/size)*100
    return acc

In [42]:
Beta,One_Hot_Encoding=KELM_Beta(sigma,c,np.array(train_features),np.array(train_labels))
p1=KELM_Output(sigma,np.array(train_features),np.array(test_features),Beta,One_Hot_Encoding)
print("test",acc(p1,test_labels))

p2=KELM_Output(sigma,np.array(train_features),np.array(train_features),Beta,One_Hot_Encoding)
print("train",acc(p2,train_labels))

# test 97.46192893401016
# train 100.0


test 97.46192893401016
train 100.0


In [43]:
test_labels1=encode(test_labels)
print(accuracy_score(test_labels1, p1))
print(classification_report(test_labels1, p1))
print(confusion_matrix(test_labels1, p1))


0.9746192893401016
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       1.00      1.00      1.00         6
           2       1.00      0.90      0.95        41
           3       1.00      1.00      1.00        15
           4       0.93      1.00      0.96        39
           5       0.96      1.00      0.98        22
           6       1.00      0.98      0.99        54

    accuracy                           0.97       197
   macro avg       0.98      0.98      0.98       197
weighted avg       0.98      0.97      0.97       197

[[20  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0]
 [ 1  0 37  0  3  0  0]
 [ 0  0  0 15  0  0  0]
 [ 0  0  0  0 39  0  0]
 [ 0  0  0  0  0 22  0]
 [ 0  0  0  0  0  1 53]]


In [44]:
#Try the model 

In [57]:
from sklearn.decomposition import PCA

img = io.imread(r'CK+48\happy\S010_006_00000013.png')
f=feature_DLTP(img)
fp= pca.transform([f])

p1=KELM_Output(sigma,np.array(train_features),np.array(fp),Beta,One_Hot_Encoding)

print ('pred label:', predictlist[p1[0]] )


pred label: happy
